In [44]:
import pandas as pd 
import os
import string
import subprocess
import time
import shutil

docspath = './documents'                        # This folder should contain the Contracts in pdf form
assert os.path.exists(docspath)
path, dirs, files = next(os.walk(docspath))
filesno = len(files)
print("Documents folder contains "+str(filesno)+" files")

Documents folder contains 11 files


### Tried and Failed: Tabula, PDFMiner.six, Tika

## Poppler - Convert all PDFs to Images

In [47]:
def convertpdf2image():
    # test poppler command
    # print('"%s" -jpeg -r %s "%s" "%s"' % (poppler_path, dpi, pdf_path, outputfile))
    
    docspath = './documents'
    poppler_path = r"C:/Users/george.gousios.SYNECHRON/Desktop/libs/poppler-0.68.0/bin/pdftoppm.exe"    # path to extracted Poppler
    dpi=300
    cfl = []                          # list of created folders! 

    for pdf_file in os.listdir(docspath):        # for every file in directory
        pdf_path = docspath+'/'+pdf_file
        print(pdf_path)                          # test full path        
        if pdf_file.endswith(".pdf"):
            filename, file_extension = os.path.splitext(pdf_path)
            outputpath = './'+filename
            if not os.path.exists(outputpath):
                os.mkdir(outputpath)
                #print("Directory " , outputpath ,  " Created ") 
            cfl.append(outputpath)
            outputfile = outputpath + '/'+os.path.splitext(pdf_file)[0]+'.jpeg'
            #assert os.path.exists(outputfile)
            assert os.path.exists(poppler_path)
            process = subprocess.Popen('"%s" -jpeg -r %s "%s" "%s"' % (poppler_path, dpi, pdf_path, outputfile))
            out, err = process.communicate()
        elif pdf_file.endswith(".tiff"):              # tiff files dont need poppler, handled all at once by TS!
            filename, file_extension = os.path.splitext(pdf_path)
            print(filename)
            outputpath = './'+filename
            if not os.path.exists(outputpath):
                #oldmask = os.umask(0o000)
                os.mkdir(outputpath, 0o777)   
                #os.umask(oldmask)
            cfl.append(outputpath)
            print("From "+str(docspath+pdf_file)+" to "+str(filename+"/"+pdf_file))
            #shutil.move(docspath+"/"+pdf_file, filename)
            shutil.copy(docspath+"/"+pdf_file, filename)
    return cfl
            
    
createdfolderlist  = convertpdf2image()

./documents/CSA example 0
From ./documentsCSA example 0.tiff to ./documents/CSA example 0/CSA example 0.tiff


In [114]:
createdfolderlist

['././documents/CSA example 0',
 '././documents/CSA example 1',
 '././documents/CSA example 2',
 '././documents/CSA example 3',
 '././documents/CSA example 4',
 '././documents/GMRA example 1',
 '././documents/GMRA example 2',
 '././documents/GMRA example 3',
 '././documents/GMSLA example 1',
 '././documents/GMSLA example 2',
 '././documents/GMSLA example 3']

## Tesseract - Extract Text and match - 1 min/doc 

In [48]:
import pytesseract as ps

def ocr(img): 
    ps.pytesseract.tesseract_cmd = r'C:\Tesseract-OCR\tesseract'             # needed if tesseract not in PATH
    text = ps.image_to_string(img)
    text = text.translate(str.maketrans('', '', string.punctuation)).lower().rstrip()    
                                                     # remove punctuation and lowercase
    return text

In [49]:
GMRA_subtext = 'EONIA in case of Cash Margin in euro'
GMSLA_subtext = 'euro: “EUR-EONIA” for a day “d” in the relevant Calculation Period, is a reference rate equal to the overnight rate as calculated by the European Central Bank on the first TARGET Settlement Day and reported on Bloomberg page “EONIA Index”'
ISDACSA_subtext = 'EONIA. “EONIA” means for any day the reference rate equal to the overnight rate as calculated by the European Central Bank and appearing on Reuters page EONIA on the first TARGET Settlement Day following that day. For the purposes of this Annex, TARGET Settlement Day means any day on which the Trans-European Automated Real-Time Gross Settlement Express Transfer (TARGET) System is open.'

GMRA_subtext = GMRA_subtext.translate(str.maketrans('', '', string.punctuation)).lower()
GMSLA_subtext = GMSLA_subtext.translate(str.maketrans('', '', string.punctuation)).lower()
ISDACSA_subtext = ISDACSA_subtext.translate(str.maketrans('', '', string.punctuation)).lower()

def_dict = {'CSA':ISDACSA_subtext, 'GMRA':GMRA_subtext, 'GMSLA':GMSLA_subtext}
def_dict

{'CSA': 'eonia “eonia” means for any day the reference rate equal to the overnight rate as calculated by the european central bank and appearing on reuters page eonia on the first target settlement day following that day for the purposes of this annex target settlement day means any day on which the transeuropean automated realtime gross settlement express transfer target system is open',
 'GMRA': 'eonia in case of cash margin in euro',
 'GMSLA': 'euro “eureonia” for a day “d” in the relevant calculation period is a reference rate equal to the overnight rate as calculated by the european central bank on the first target settlement day and reported on bloomberg page “eonia index”'}

In [50]:
# from fuzzywuzzy import fuzz

# scoresdict = {0:'CSA', 1:'GMRA', 2:'GMSLA'}

# # csa 2: 'equal' does not exist in text
# # csa 2: 'provision' vs 'annex' settlement
# # csa 2: on 'the' reuters page

# def getscores(pagetext):
#     gmra_score = fuzz.token_set_ratio(pagetext, GMRA_subtext)
#     gmsla_score = fuzz.token_set_ratio(pagetext, GMSLA_subtext)
#     isdacsa_score = fuzz.token_set_ratio(pagetext, ISDACSA_subtext)
#     return [isdacsa_score, gmra_score, gmsla_score]

# def classifypdf(pdfname):
#     eonialist = []
#     filename, file_extension = os.path.splitext(pdfname)
#     pdfimages = './documents/'+str(filename)
#     #print(pdfimages)
#     #print('\n')
#     maxscore = -1
#     maxpage = -1
#     count = 0
#     for image in os.listdir(pdfimages):                       # for every page (image) of pdf
#         count = count+1                                       # keep current page number
#         image_path = pdfimages+'/'+image
#         pagetext = ocr(image_path)                            # get text from image (lowercased + punc. removed)
#         scoreslist = getscores(pagetext)                      # get a list of match 3 scores for each class   
#         if 'eonia' in pagetext:                               # check for any reference of 'eonia' in the page
#             #print('Found EONIA in page'+str(count))
#             eonialist.append(count)
        
#         #print("Page "+str(count)+", scores: CSA, GMRA, GMSLA")
#         #print(str(scoreslist))
#         maxlistelement = max(scoreslist)
#         maxlistposition = scoreslist.index(maxlistelement)
#         #print("Maxposition is "+str(maxlistposition))
#         maxlistcategory = scoresdict[maxlistposition]
#         #print("MaxlistCategory: "+str(maxlistcategory))
#         if maxlistelement>maxscore:
#             maxscore = maxlistelement
#             maxcategory = maxlistcategory
#             maxpage = count  
#             #print("New rec: "+str(maxscore)+" on "+str(maxcategory))
#             if maxscore>90:
#                 return 'True', maxcategory, maxscore, maxpage, eonialist        
#     return 'False', maxcategory, maxscore, maxpage, eonialist

# def trueclass(name):
#     parts = name.split(' ')
#     docClass = parts[0]
#     return parts[0]

In [145]:
### new stuff

from fuzzywuzzy import fuzz

# scoresdict = {0:'CSA', 1:'GMRA', 2:'GMSLA'}

# csa 2: 'equal' does not exist in text
# csa 2: 'provision' vs 'annex' settlement
# csa 2: on 'the' reuters page

def getscore(pagetext, docclass):
    return fuzz.token_set_ratio(pagetext, def_dict[docclass])            # needs change
    #return fuzz.partial_ratio(pagetext, def_dict[docclass])             # maybe better?
    

def getexactmatch(pagetext, docclass):    
    return def_dict[docclass] in pagetext

def classifypdf(pdfname):
    eonialist = []
    filename, file_extension = os.path.splitext(pdfname)
    pdfimages = './documents/'+str(filename)
    
    docclass = trueclass(pdfname)
    print(pdfimages)
    #print('\n')
    maxscore = -1
    maxpage = -1
    count = 0
    for image in os.listdir(pdfimages):                       # for every page (image) of pdf
        eoniaflag = 0                                         # initialize eonia flag for every page
        count = count+1                                       # keep current page number
        image_path = pdfimages+'/'+image
        pagetext = ocr(image_path)                            # get text from image (lowercased + punc. removed)
        score = getscore(pagetext,docclass)                   # get a match score for the document class definiton
        exactmatch = getexactmatch(pagetext,docclass)         # get an exact match (True/False)
        if 'eonia' in pagetext:                               # check for any reference of 'eonia' in the page
            eoniaflag = 1
            #print('Found EONIA in page'+str(count))
            eonialist.append(count)
        
        if score>maxscore:
            maxscore = score
            maxpage = count  
            print("\tNew rec: "+str(maxscore)+" on page "+str(count))
            if maxscore>90 & eoniaflag==1:
                return docclass, exactmatch, maxscore, maxpage, eonialist        
    return docclass, exactmatch, maxscore, maxpage, eonialist

def trueclass(name):
    parts = name.split(' ')
    docClass = parts[0]
    return parts[0]

## 8m for 10 documents ~ 40h for 3000 documents

In [146]:
# Single Threaded execution takes 8 minutes

def performTask():
    df = pd.DataFrame()
    df['Contracts'] = [pdf_file for pdf_file in os.listdir(docspath) 
                       if pdf_file.endswith(".pdf") or pdf_file.endswith(".tiff")]    
    #df = df.head(7)   
    #df = df.iloc[6:]   # only 4th document
    
    df['TrueClass'], , df['ExactMatch'], df['MatchPercentage'], df['PageNo'], df['EoniaRefPages'] = zip(*df['Contracts'].map(classifypdf))
    return df



start = time.time()

df = performTask()

end = time.time()
print("Task took "+str(float(end - start)/60)+" minutes to complete")
df

./documents/CSA example 0
New rec: 83 on page 1
./documents/CSA example 1
New rec: 51 on page 1
New rec: 56 on page 2
New rec: 62 on page 4
./documents/CSA example 2
New rec: 38 on page 1
New rec: 48 on page 2
New rec: 58 on page 4
New rec: 98 on page 5
./documents/CSA example 3
New rec: 47 on page 1
New rec: 53 on page 2
New rec: 55 on page 4
./documents/CSA example 4
New rec: 56 on page 1
New rec: 58 on page 2
New rec: 73 on page 16
./documents/GMRA example 1
New rec: 26 on page 1
New rec: 80 on page 2
New rec: 82 on page 3
./documents/GMRA example 2
New rec: 26 on page 1
New rec: 80 on page 2
./documents/GMRA example 3
New rec: 47 on page 1
New rec: 90 on page 2
New rec: 100 on page 27
./documents/GMSLA example 1
New rec: 35 on page 1
New rec: 44 on page 3
New rec: 47 on page 4
New rec: 60 on page 6
./documents/GMSLA example 2
New rec: 36 on page 1
New rec: 54 on page 2
New rec: 56 on page 6
New rec: 84 on page 25
./documents/GMSLA example 3
New rec: 32 on page 1
New rec: 40 on page

,Contracts,TrueClass,MatchPercentage,PageNo,ExactMatch,EoniaRefPages
0,CSA example 0.tiff,CSA,83,1,False,[]
1,CSA example 1.pdf,CSA,62,4,False,[]
2,CSA example 2.pdf,CSA,98,5,False,[5]
3,CSA example 3.pdf,CSA,55,4,False,[5]
4,CSA example 4.pdf,CSA,73,16,False,[16]
5,GMRA example 1.pdf,GMRA,82,3,False,[3]
6,GMRA example 2.pdf,GMRA,80,2,False,[]
7,GMRA example 3.pdf,GMRA,100,27,False,[27]
8,GMSLA example 1.pdf,GMSLA,60,6,False,[]
9,GMSLA example 2.pdf,GMSLA,84,25,False,"[25, 27]"


In [147]:
def setMatch(row):
    pageno = row['PageNo']
    eoniarefpages = row['EoniaRefPages']
    matchperc = row['MatchPercentage']
    if pageno in eoniarefpages and int(matchperc)>90:
        return True
    else:
        return False

#df[['a', 'b']].apply(myfunction2, axis=1) 

df['Match'] =  df[['PageNo', 'EoniaRefPages', 'MatchPercentage']].apply(setMatch, axis=1)         
df.head(10)

,Contracts,TrueClass,MatchPercentage,PageNo,ExactMatch,EoniaRefPages,Match
0,CSA example 0.tiff,CSA,83,1,False,[],False
1,CSA example 1.pdf,CSA,62,4,False,[],False
2,CSA example 2.pdf,CSA,98,5,False,[5],True
3,CSA example 3.pdf,CSA,55,4,False,[5],False
4,CSA example 4.pdf,CSA,73,16,False,[16],False
5,GMRA example 1.pdf,GMRA,82,3,False,[3],False
6,GMRA example 2.pdf,GMRA,80,2,False,[],False
7,GMRA example 3.pdf,GMRA,100,27,False,[27],True
8,GMSLA example 1.pdf,GMSLA,60,6,False,[],False
9,GMSLA example 2.pdf,GMSLA,84,25,False,"[25, 27]",False


In [148]:
df.to_csv('./results_new.csv', index=False)

## Delete generated images

In [69]:
import shutil

for file in os.listdir(docspath):    # delete all generated folders
    if not file.endswith(".pdf"):
        folderpath = str(docspath)+'/'+str(file)
        print(folderpath + " deleted")
        shutil.rmtree(folderpath)

## Open results file

In [32]:
df = pd.read_csv('./results_new.csv')
df

,Contracts,Match,TrueClass,MatchPercentage,PageNo,EoniaRefPages,FinalPage
0,CSA example 1.pdf,False,CSA,62,4,[],-1
1,CSA example 2.pdf,True,CSA,98,5,[5],5
2,CSA example 3.pdf,False,CSA,55,4,[5],-1
3,CSA example 4.pdf,False,CSA,73,16,[16],16
4,GMRA example 1.pdf,False,GMRA,82,3,[3],3
5,GMRA example 2.pdf,False,GMRA,80,2,[],-1
6,GMRA example 3.pdf,True,GMRA,100,27,[27],27
7,GMSLA example 1.pdf,False,GMSLA,60,6,[],-1
8,GMSLA example 2.pdf,False,GMSLA,84,25,"[25, 27]",25
9,GMSLA example 3.pdf,False,GMSLA,46,4,[],-1


# MISC

## 1 Choosing the right function

In [125]:
# ratio: self-explainable                                     
# partial_ratio: compares partial string similarity.
# token_sort_ratio: ignores word order
# token_set_ratio: ignores duplicated words

s1 = "Catherine Gitau"
s2 = "BLA BLA BLA Catherine Gitau BLA BLA BLA"
s3 = "BLA Catherine DE LA BLA BLA Gitau BLA BLA BLA ELEOS MLK TI EINAI AUTO"

print("Ratio:")
print("\t"+str(fuzz.ratio(s1, s2)))                # nope - 1 and 2 are too far
print("\t"+str(fuzz.ratio(s1, s3)))
print("Token_Set_Ratio:")
print("\t"+str(fuzz.token_set_ratio(s1, s2)))      # nope - 1 and 3 are too close - that's what we currently have
print("\t"+str(fuzz.token_set_ratio(s1, s3)))
print("Partial_Ratio:")
print("\t"+str(fuzz.partial_ratio(s1, s2)))        # maybe better?
print("\t"+str(fuzz.partial_ratio(s1, s3)))
print("Token_Sort_Ratio:")
print("\t"+str(fuzz.token_sort_ratio(s1, s2)))     # nope - 1 and 2 are too far
print("\t"+str(fuzz.token_sort_ratio(s1, s3)))

Ratio:
	56
	36
Token_Set_Ratio:
	100
	100
Partial_Ratio:
	100
	67
Token_Sort_Ratio:
	56
	36


## 2 Exact Matching:

In [143]:
s1 = 'TIVUZARESEINAIAUTES gamw to spitaki mou'
s2 = 'KAI POY LES VLEPW ENAN MUSTH KAI ME RWTAEI TIVUZARESEINAIAUTES gamw to spitaki mou, KAI TOU LEW E PES M ESU'

print(s1 in s2)

True
